# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [7]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [8]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print (classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [9]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [10]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i in range(len(classes)):
    data, target = load_pose_data(i)
    print ('data: ' + str(data))
    all_data.extend(data)
    all_target.extend(target)

print ('total number of data', len(all_data))

5689315795898], [0.10588788986206055, 0.3405900001525879, -0.4893040657043457, -0.13196587562561035, 0.10588788986206055, 0.3559298515319824, -0.6750020980834961, 0.052197933197021484, 1.3159785270690918, 0.09075689315795898], [0.10588788986206055, 0.33905601501464844, -0.49083805084228516, -0.13196587562561035, 0.10588788986206055, 0.3559298515319824, -0.6750020980834961, 0.052197933197021484, 1.3159785270690918, 0.08901166915893555], [0.10588788986206055, 0.33905601501464844, -0.49083805084228516, -0.13196587562561035, 0.10588788986206055, 0.3559298515319824, -0.6750020980834961, 0.052197933197021484, 1.3177237510681152, 0.08901166915893555], [-0.1579599380493164, -0.3251659870147705, -0.7562201023101807, 2.083130121231079, -0.15949392318725586, 0.10895586013793945, -1.540177822113037, 2.1353702545166016, 1.4346609115600586, 0.1099553108215332]]
data: [[-0.7562201023101807, 0.7839159965515137, -1.630600094795227, 0.2607381343841553, -0.7562201023101807, 0.09975194931030273, -1.570858

In [11]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]
test_data = permutation[n_training_data:]
all_data = np.asarray(all_data)
all_target = np.asarray(all_target)

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [12]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [13]:
clf.fit(all_data[training_data], all_target[training_data])

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### predicting

In [14]:
clf.predict(all_data[-1:]), all_target[-1:]

(array([10]), array([10]))

In [15]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [82]:
expected = []
predicted = []
# YOUR CODE HERE

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         5
          1       1.00      1.00      1.00         7
          2       1.00      1.00      1.00        15
          3       1.00      1.00      1.00         5
          4       1.00      1.00      1.00        20
          5       1.00      1.00      1.00         6
          6       1.00      1.00      1.00         8
          7       1.00      1.00      1.00         7
          8       1.00      1.00      1.00        18
          9       1.00      1.00      1.00         8
         10       1.00      1.00      1.00        41

avg / total       1.00      1.00      1.00       140


Confusion matrix:
[[ 5  0  0  0  0  0  0  0  0  0  0]
 [ 0  7  0  0  0  0  0  0  0  0  0]
 [ 0  0 15  0  0  0  0  0  0  0  0]
 [ 0  0  0  5  0  0  0  0  0  0  0]
 [ 0  0  0  0 20  0  0  0  0  0  0]
 [ 0  0  0  0  0  6  0  0  0  0  0]
 [ 0  0  0  0  0  0  8  0  0  0  0]
 [ 0  0  0  0 

## 4. Evaluate on the test data

In [16]:
expected = all_target[test_data]
predicted = clf.predict(all_data[test_data])
# YOUR CODE HERE

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         5
           2       0.90      1.00      0.95         9
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         4
           5       1.00      1.00      1.00         5
           6       1.00      1.00      1.00         7
           7       1.00      0.67      0.80         3
           8       1.00      1.00      1.00         4
           9       1.00      1.00      1.00         3
          10       1.00      1.00      1.00        19

   micro avg       0.99      0.99      0.99        67
   macro avg       0.99      0.97      0.98        67
weighted avg       0.99      0.99      0.98        67


Confusion matrix:
[[ 5  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  0  0  0  0  0  0  0]
 [ 0  0  9  0  0  0  0  0  0  0  0]
 [ 0  0  0  3  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [17]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [18]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1].reshape(1, -1)), all_target[-1]

(array([10]), 10)